In [18]:
import torch
import pandas as pd
import numpy as np
import random
import os
from google.colab import drive

In [19]:
# ========================================================
# 🚨 [통일 필수] 경로 및 시드 설정
# ========================================================

# 구글 드라이브 마운트
drive.mount('/content/drive')

MODULE_PATH = "/content/drive/MyDrive/2025CV"
CSV_PATH_FULL = os.path.join(MODULE_PATH, "out_datas_filtered.csv") # 원본 분할 정보 CSV
FINAL_CSV_NAME = "meta_c2s_10_2_2_sampling_ID.csv" # 새로운 파일명

# --- 샘플링 비율 (ID 개수 기준) ---
TRAIN_ID_RATE = 0.10  # 훈련 ID 10%
VAL_TEST_ID_RATE = 0.02 # 검증/테스트 ID 2%
EXPERIMENT_SEED = 42

# 🚨 시드 고정
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(EXPERIMENT_SEED)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# ========================================================
# 2. ✂️ ID 기반 필터링 함수 정의
# ========================================================

def filter_by_id_sample(df_split, sample_rate, split_name, seed):
    """
    DataFrame 내의 고유 item_id 중 sample_rate만큼만 무작위로 선택하여 DataFrame을 필터링합니다.
    """
    # 1. 해당 split의 고유 ID 목록 추출
    unique_ids = df_split['item_id'].unique()

    # 2. 샘플링할 ID 개수 계산
    n_sample_ids = max(1, int(len(unique_ids) * sample_rate))

    # 3. 🚨 ID 목록 무작위 샘플링 (np.random.choice 사용)
    #    random_state를 사용하여 재현성 확보
    np.random.seed(seed)
    sampled_ids = np.random.choice(unique_ids, size=n_sample_ids, replace=False)

    # 4. 샘플링된 ID를 가진 모든 Pairs 보존
    filtered_df = df_split[df_split['item_id'].isin(sampled_ids)].reset_index(drop=True)

    print(f"✅ {split_name} - Original IDs: {len(unique_ids):,}, Sampled IDs: {len(sampled_ids):,}")

    return filtered_df


In [21]:
# ========================================================
# 3. 🏃 샘플링 실행 및 통합
# ========================================================

# 3-1. 데이터 로드 및 초기 분리
df_full = pd.read_csv(CSV_PATH_FULL)

train_df = df_full[df_full['split'] == 'train'].copy()
val_df = df_full[df_full['split'] == 'val'].copy()
test_df = df_full[df_full['split'] == 'test'].copy()

# 3-2. ID 필터링 적용
sampled_train_df = filter_by_id_sample(train_df, TRAIN_ID_RATE, 'Train', EXPERIMENT_SEED)
sampled_val_df = filter_by_id_sample(val_df, VAL_TEST_ID_RATE, 'Validation', EXPERIMENT_SEED)
sampled_test_df = filter_by_id_sample(test_df, VAL_TEST_ID_RATE, 'Test', EXPERIMENT_SEED)



✅ Train - Original IDs: 12,214, Sampled IDs: 1,221
✅ Validation - Original IDs: 6,235, Sampled IDs: 124
✅ Test - Original IDs: 6,134, Sampled IDs: 122


In [22]:
# 4. 🔗 최종 CSV 통합 및 저장
df_final_lightweight = pd.concat([sampled_train_df, sampled_val_df, sampled_test_df])

# 5. 최종 확인 및 저장
print("\n================== 최종 ID 샘플링 결과 ==================")
print(f"✅ 최종 Train Pair 수 (10% ID): {len(sampled_train_df):,}")
print(f"✅ 최종 Validation Pair 수 (2% ID): {len(sampled_val_df):,}")
print(f"✅ 최종 Test Pair 수 (2% ID): {len(sampled_test_df):,}")
print(f"✅ 총 Pair 수: {len(df_final_lightweight):,}")

# 🚨 최종 CSV 파일로 저장
FINAL_CSV_PATH = os.path.join(MODULE_PATH, FINAL_CSV_NAME)
df_final_lightweight.to_csv(FINAL_CSV_PATH, index=False)

print(f"\n🎉 최종 ID 샘플링 CSV 파일 생성 완료. 경로:\n{FINAL_CSV_PATH}")


================== 최종 ID 샘플링 결과 ==================
✅ 최종 Train Pair 수 (10% ID): 12,421
✅ 최종 Validation Pair 수 (2% ID): 1,097
✅ 최종 Test Pair 수 (2% ID): 956
✅ 총 Pair 수: 14,474

🎉 최종 ID 샘플링 CSV 파일 생성 완료. 경로:
/content/drive/MyDrive/2025CV/meta_c2s_10_2_2_sampling_ID.csv
